In [1]:
from dask.distributed import Client
import dask
import dask.array as da
from dask import delayed
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pyextremes as pe
from pyextremes import EVA
import glob
import pandas as pd
from datetime import timedelta
import datetime
from datetime import datetime as dt
from scipy.stats import genextreme as gev
from scipy.stats import genpareto as gp
import scipy.stats as stats
client = Client("tcp://127.0.0.1:43523")

In [3]:
input_file = '/home/davidbyrne/disks/ssd200/test.nc'
#dataset = xr.open_zarr(input_file).chunk({'year':-1})
dataset = xr.open_dataset(input_file, chunks={})
data = dataset.Twb.load().data
data = da.from_array(data, chunks=[20, 20, -1])

In [4]:
def fit_gev_model(chunk):
    n_r, n_c, n_t = chunk.shape
    n_pts = n_r*n_c
    chunkF = chunk.reshape((n_pts, n_t))
    params = np.zeros((n_pts, 4))*np.nan
    
    for pt in range(n_pts):
        extremes = chunkF[pt, :]
        extremes = extremes[~np.isnan(extremes)]
        if len(extremes) < 50:
            continue
        ff = gev.fit(extremes)
        params[pt, :3] = ff
        
        # Kolmogorov-Smirnov test for fitted distribution
        ks = stats.kstest(extremes, 'genextreme', args=ff)
        params[pt, 3] = ks[1]

    return params.reshape((n_r, n_c, 4))
        
def AIC(data, model, params):
    k = len(params)
    logLik = np.sum( model.logpdf(data, params[0], params[1], params[2]) ) 
    return 2*k - 2*logLik

In [5]:
%%time
mapped = da.map_blocks(fit_gev_model, data)
mapped = mapped.compute()
ds = xr.Dataset(data_vars=dict(
                    parameters = (['lat','lon','param'], mapped)))
ds.to_netcdf('/home/davidbyrne/disks/ssd200/test2.nc')

CPU times: user 986 ms, sys: 257 ms, total: 1.24 s
Wall time: 24min 8s
